In [54]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models
from sklearn.metrics import f1_score
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_text as text
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import models, layers, callbacks, optimizers
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [3]:
df_narratives = pd.read_csv('train_features_X4juyT6.csv')
df_features = pd.read_csv('train_labels_JxtENGl.csv')
print("Columns in df_narratives:", df_narratives.columns.tolist())
print("Columns in df_features:", df_features.columns.tolist())

merged_df = pd.merge(df_narratives, df_features, on='uid', how='inner')

Columns in df_narratives: ['uid', 'NarrativeLE', 'NarrativeCME']
Columns in df_features: ['uid', 'DepressedMood', 'MentalIllnessTreatmentCurrnt', 'HistoryMentalIllnessTreatmnt', 'SuicideAttemptHistory', 'SuicideThoughtHistory', 'SubstanceAbuseProblem', 'MentalHealthProblem', 'DiagnosisAnxiety', 'DiagnosisDepressionDysthymia', 'DiagnosisBipolar', 'DiagnosisAdhd', 'IntimatePartnerProblem', 'FamilyRelationship', 'Argument', 'SchoolProblem', 'RecentCriminalLegalProblem', 'SuicideNote', 'SuicideIntentDisclosed', 'DisclosedToIntimatePartner', 'DisclosedToOtherFamilyMember', 'DisclosedToFriend', 'InjuryLocationType', 'WeaponType1']


In [4]:
nltk.download('stopwords')
nltk.download('punkt')
def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = text.translate(str.maketrans("", "", string.punctuation))
        tokens = word_tokenize(text)
        stop_words = set(stopwords.words('english'))
        filtered_tokens = [word for word in tokens if word not in stop_words]
        return ' '.join(filtered_tokens)  # Join back to string for BERT
    return ""

merged_df['NarrativeLE'] = merged_df['NarrativeLE'].apply(preprocess_text)
merged_df['NarrativeCME'] = merged_df['NarrativeCME'].apply(preprocess_text)

# Check for null values
print(merged_df.isnull().sum())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\REX\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\REX\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


uid                             0
NarrativeLE                     0
NarrativeCME                    0
DepressedMood                   0
MentalIllnessTreatmentCurrnt    0
HistoryMentalIllnessTreatmnt    0
SuicideAttemptHistory           0
SuicideThoughtHistory           0
SubstanceAbuseProblem           0
MentalHealthProblem             0
DiagnosisAnxiety                0
DiagnosisDepressionDysthymia    0
DiagnosisBipolar                0
DiagnosisAdhd                   0
IntimatePartnerProblem          0
FamilyRelationship              0
Argument                        0
SchoolProblem                   0
RecentCriminalLegalProblem      0
SuicideNote                     0
SuicideIntentDisclosed          0
DisclosedToIntimatePartner      0
DisclosedToOtherFamilyMember    0
DisclosedToFriend               0
InjuryLocationType              0
WeaponType1                     0
dtype: int64


In [9]:
narrative_le = merged_df['NarrativeLE'].tolist()
narrative_cme = merged_df['NarrativeCME'].tolist()

In [10]:
bert_preprocess = tf.saved_model.load('D:/MLProjects/YMHComp/submission_src/assets/bert_preprocess')
bert_encoder = tf.saved_model.load('D:/MLProjects/YMHComp/submission_src/assets/bert_encoder')

def get_sentence_embedding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

def get_batch_embeddings(sentences, batch_size=32):
    embeddings = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]
        embeddings.append(get_sentence_embedding(batch))
    return tf.concat(embeddings, axis=0)

narrative_le_embeddings = get_batch_embeddings(narrative_le)
narrative_cme_embeddings = get_batch_embeddings(narrative_cme)

narrative_le_embeddings_df = pd.DataFrame(narrative_le_embeddings.numpy())
narrative_cme_embeddings_df = pd.DataFrame(narrative_cme_embeddings.numpy())

merged_df = pd.concat([merged_df, narrative_le_embeddings_df.add_prefix('narrative_le_')], axis=1)
merged_df = pd.concat([merged_df, narrative_cme_embeddings_df.add_prefix('narrative_cme_')], axis=1)

In [2]:
X = merged_df[[col for col in merged_df.columns if 'narrative_le_' in col or 'narrative_cme_' in col]]
y = merged_df[['DepressedMood', 'MentalIllnessTreatmentCurrnt', 'HistoryMentalIllnessTreatmnt', 
               'SuicideAttemptHistory', 'SuicideThoughtHistory', 'SubstanceAbuseProblem', 
               'MentalHealthProblem', 'DiagnosisAnxiety', 'DiagnosisDepressionDysthymia', 
               'DiagnosisBipolar', 'DiagnosisAdhd', 'IntimatePartnerProblem', 
               'FamilyRelationship', 'Argument', 'SchoolProblem', 
               'RecentCriminalLegalProblem', 'SuicideNote', 
               'SuicideIntentDisclosed', 'DisclosedToIntimatePartner', 
               'DisclosedToOtherFamilyMember', 'DisclosedToFriend']]
combined_embeddings = tf.concat([narrative_le_embeddings, narrative_cme_embeddings], axis=1)
input_shape = combined_embeddings.shape[1] 

NameError: name 'merged_df' is not defined

In [69]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Define the model
input_shape = X_train_scaled.shape[1]
model = models.Sequential([
    layers.Input(shape=(input_shape,)),
    layers.Dense(512),
    layers.LeakyReLU(alpha=0.2),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256),
    layers.LeakyReLU(alpha=0.2),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(y.shape[1], activation='sigmoid')  # assuming binary classification
])

# Compile the model
optimizer = optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')
lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model
history = model.fit(
    X_train_scaled, y_train,
    epochs=100, batch_size=32,
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping, model_checkpoint, lr_scheduler]
)

# Load the best model after training
model.load_weights('best_model.h5')

# Make predictions
y_pred = model.predict(X_val_scaled)

Epoch 1/100
100/100 [==============================] - 2s 13ms/step - loss: 0.8752 - accuracy: 0.0534 - val_loss: 0.6960 - val_accuracy: 0.0763 - lr: 1.0000e-04
Epoch 2/100
100/100 [==============================] - 1s 10ms/step - loss: 0.8041 - accuracy: 0.0547 - val_loss: 0.6739 - val_accuracy: 0.0700 - lr: 1.0000e-04
Epoch 3/100
100/100 [==============================] - 1s 11ms/step - loss: 0.7693 - accuracy: 0.0572 - val_loss: 0.6486 - val_accuracy: 0.0750 - lr: 1.0000e-04
Epoch 4/100
100/100 [==============================] - 1s 10ms/step - loss: 0.7335 - accuracy: 0.0594 - val_loss: 0.6232 - val_accuracy: 0.0763 - lr: 1.0000e-04
Epoch 5/100
100/100 [==============================] - 1s 11ms/step - loss: 0.7072 - accuracy: 0.0728 - val_loss: 0.6064 - val_accuracy: 0.0875 - lr: 1.0000e-04
Epoch 6/100
100/100 [==============================] - 1s 10ms/step - loss: 0.6836 - accuracy: 0.0747 - val_loss: 0.5913 - val_accuracy: 0.0862 - lr: 1.0000e-04
Epoch 7/100
100/100 [=============

In [40]:
#injury prediction
X = merged_df[[col for col in merged_df.columns if col.startswith('narrative')]]
y_injury_location = merged_df['InjuryLocationType']

# Encode labels to be in range [0, num_classes - 1]
label_encoder = LabelEncoder()
y_injury_location_encoded = label_encoder.fit_transform(y_injury_location)

# Split data into training and validation sets
X_train, X_val, y_injury_train, y_injury_val = train_test_split(
    X, y_injury_location_encoded, test_size=0.2, random_state=42
)

class_weights = compute_class_weight('balanced', classes=np.unique(y_injury_location_encoded), y=y_injury_location_encoded)
class_weights_dict = dict(enumerate(class_weights))

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Define the model
input_shape = X_train_scaled.shape[1]
num_classes = len(np.unique(y_injury_location_encoded))
model_injury = models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(input_shape,), kernel_regularizer='l2'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu', kernel_regularizer='l2'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu', kernel_regularizer='l2'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
optimizer = optimizers.Adam(learning_rate=0.001)
model_injury.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('best_injury_model.h5', save_best_only=True, monitor='val_loss')
lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model
history = model_injury.fit(
    X_train_scaled, y_injury_train, 
    epochs=100, batch_size=64, 
    validation_data=(X_val_scaled, y_injury_val),
    class_weight=class_weights_dict,
    callbacks=[early_stopping, model_checkpoint, lr_scheduler]
)

# Load the best model
model_injury.load_weights('best_injury_model.h5')

# Make predictions
y_injury_pred = np.argmax(model_injury.predict(X_val_scaled), axis=1)

Epoch 1/100
50/50 [==============================] - 2s 30ms/step - loss: 14.5501 - accuracy: 0.1597 - val_loss: 13.1354 - val_accuracy: 0.3038 - lr: 0.0010
Epoch 2/100
50/50 [==============================] - 1s 15ms/step - loss: 12.3553 - accuracy: 0.1875 - val_loss: 11.0536 - val_accuracy: 0.2763 - lr: 0.0010
Epoch 3/100
50/50 [==============================] - 1s 15ms/step - loss: 10.2645 - accuracy: 0.1966 - val_loss: 9.3911 - val_accuracy: 0.2350 - lr: 0.0010
Epoch 4/100
50/50 [==============================] - 1s 15ms/step - loss: 8.5441 - accuracy: 0.2197 - val_loss: 7.8335 - val_accuracy: 0.1675 - lr: 0.0010
Epoch 5/100
50/50 [==============================] - 1s 15ms/step - loss: 7.3327 - accuracy: 0.2156 - val_loss: 6.7563 - val_accuracy: 0.2325 - lr: 0.0010
Epoch 6/100
50/50 [==============================] - 1s 16ms/step - loss: 6.2682 - accuracy: 0.2231 - val_loss: 5.9679 - val_accuracy: 0.1875 - lr: 0.0010
Epoch 7/100
50/50 [==============================] - 1s 16ms/step

In [41]:
f1_injury = f1_score(y_injury_val, y_injury_pred, average='micro')
print("F1 Score Injury (weighted):", f1_injury)

F1 Score Injury (weighted): 0.65


In [71]:
X = merged_df[[col for col in merged_df.columns if col.startswith('narrative')]]
y_weapon = merged_df['InjuryLocationType']
y_weapon = y_weapon - 1 

# Split data into training and validation sets
X_train, X_val, y_weapon_train, y_weapon_val = train_test_split(
    X, y_weapon, test_size=0.2, random_state=42
)

# Apply scaling to features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Define the model
input_shape = X_train_scaled.shape[1]
model_weapon = models.Sequential([
    layers.Dense(256, activation='relu', input_shape=(input_shape,), kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(len(np.unique(y_weapon)), activation='softmax')  # Ensure this matches the number of classes
])

# Compile the model
optimizer = optimizers.Adam(learning_rate=0.001)
model_weapon.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_weapon), y=y_weapon)
class_weights_dict = dict(enumerate(class_weights))

# Callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('best_weapon_model.h5', save_best_only=True, monitor='val_loss')
lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5)

# Train the model
history = model_weapon.fit(
    X_train_scaled, y_weapon_train, 
    epochs=100, batch_size=64, 
    validation_data=(X_val_scaled, y_weapon_val),
    class_weight=class_weights_dict,
    callbacks=[early_stopping, model_checkpoint, lr_scheduler]
)

# Load the best model
model_weapon.load_weights('best_weapon_model.h5')

# Make predictions
y_weapon_pred = np.argmax(model_weapon.predict(X_val_scaled), axis=1)

Epoch 1/100
50/50 [==============================] - 1s 14ms/step - loss: 8.4284 - accuracy: 0.1759 - val_loss: 7.8899 - val_accuracy: 0.2025 - lr: 0.0010
Epoch 2/100
50/50 [==============================] - 0s 9ms/step - loss: 7.3585 - accuracy: 0.1806 - val_loss: 6.7910 - val_accuracy: 0.2288 - lr: 0.0010
Epoch 3/100
50/50 [==============================] - 0s 9ms/step - loss: 6.5456 - accuracy: 0.2266 - val_loss: 6.0198 - val_accuracy: 0.2300 - lr: 0.0010
Epoch 4/100
50/50 [==============================] - 0s 9ms/step - loss: 5.7850 - accuracy: 0.2253 - val_loss: 5.5018 - val_accuracy: 0.2175 - lr: 0.0010
Epoch 5/100
50/50 [==============================] - 0s 10ms/step - loss: 5.1125 - accuracy: 0.2472 - val_loss: 4.7328 - val_accuracy: 0.2825 - lr: 0.0010
Epoch 6/100
50/50 [==============================] - 0s 9ms/step - loss: 4.5550 - accuracy: 0.2534 - val_loss: 4.3274 - val_accuracy: 0.2887 - lr: 0.0010
Epoch 7/100
50/50 [==============================] - 0s 9ms/step - loss: 4

In [74]:
def build_model(input_shape, num_classes, dropout_rate=0.2, l2_rate=0.005):
    model = models.Sequential([
        layers.Dense(512, activation='relu', input_shape=(input_shape,), kernel_regularizer=tf.keras.regularizers.l2(l2_rate)),
        layers.BatchNormalization(),
        layers.Dropout(dropout_rate),
        layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_rate)),
        layers.BatchNormalization(),
        layers.Dropout(dropout_rate),
        layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_rate)),
        layers.BatchNormalization(),
        layers.Dropout(dropout_rate),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def train_and_evaluate(X_train, X_val, y_train, y_val, class_weights, model_save_path, num_classes):
    # Build and compile model
    model = build_model(input_shape=X_train.shape[1], num_classes=num_classes)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.0005), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

    # Callbacks
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model_checkpoint = callbacks.ModelCheckpoint(model_save_path, save_best_only=True, monitor='val_loss')
    lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=150, batch_size=64,
        validation_data=(X_val, y_val),
        class_weight=class_weights,
        callbacks=[early_stopping, model_checkpoint, lr_scheduler]
    )

    # Load best model
    model.load_weights(model_save_path)
    
    # Predict and evaluate F1 score
    y_pred = np.argmax(model.predict(X_val), axis=1)
    f1 = f1_score(y_val, y_pred, average='micro')
    print(f"F1 Score (micro): {f1}")
    return f1

# Use the function with respective inputs
f1_injury = train_and_evaluate(X_train_scaled, X_val_scaled, y_injury_train, y_injury_val, class_weights_dict, 'best_injury_model.h5', num_classes)

Epoch 1/150
50/50 [==============================] - 2s 21ms/step - loss: 8.6759 - accuracy: 0.1697 - val_loss: 8.2511 - val_accuracy: 0.2062 - lr: 5.0000e-04
Epoch 2/150
50/50 [==============================] - 1s 16ms/step - loss: 8.0814 - accuracy: 0.2262 - val_loss: 7.9509 - val_accuracy: 0.2175 - lr: 5.0000e-04
Epoch 3/150
50/50 [==============================] - 1s 15ms/step - loss: 7.7001 - accuracy: 0.2463 - val_loss: 7.6734 - val_accuracy: 0.2612 - lr: 5.0000e-04
Epoch 4/150
50/50 [==============================] - 1s 17ms/step - loss: 7.3718 - accuracy: 0.2788 - val_loss: 7.3773 - val_accuracy: 0.2562 - lr: 5.0000e-04
Epoch 5/150
50/50 [==============================] - 1s 15ms/step - loss: 7.0002 - accuracy: 0.2856 - val_loss: 7.2198 - val_accuracy: 0.2188 - lr: 5.0000e-04
Epoch 6/150
50/50 [==============================] - 1s 15ms/step - loss: 6.6380 - accuracy: 0.3125 - val_loss: 6.8614 - val_accuracy: 0.2950 - lr: 5.0000e-04
Epoch 7/150
50/50 [===========================

In [72]:
from sklearn.metrics import f1_score

# Convert predictions to binary (0 or 1) based on a threshold, typically 0.5 for sigmoid output
y_pred_binary = (y_pred > 0.5).astype(int)

f1_bin = f1_score(y_val, y_pred_binary, average='micro')
print("F1 Score Binary (weighted):", f1_bin)

f1_injury = f1_score(y_injury_val, y_injury_pred, average='micro')
print("F1 Score Injury (weighted):", f1_injury)

f1_weapon = f1_score(y_weapon_val, y_weapon_pred, average='micro')
print("F1 Score Weapon(weighted):", f1_weapon)

total_f1 = f1_bin + f1_injury + f1_weapon
total_f1 = total_f1/3
print(total_f1)

F1 Score Binary (weighted): 0.44018643190056966
F1 Score Injury (weighted): 0.65
F1 Score Weapon(weighted): 0.58125
0.5571454773001899
